### Using an OpenAI API key 

In [1]:
from langchain.llms import OpenAI

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key_env = os.environ.get("OPENAI_API_KEY")

In [3]:
openai_llm = OpenAI(openai_api_key=openai_api_key_env,model_name="gpt-3.5-turbo-instruct",temperature=0.6)

d:\Langchain\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [4]:
text = "Give a description of an apple in 25 words"

print(openai_llm.predict(text))

d:\Langchain\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




A round fruit with a smooth, shiny skin ranging from green to red. Its flesh is crisp and juicy with a sweet or tart flavor. 


### Using a HuggingFace access token to use Open Source LLM Model

In [5]:
load_dotenv()
hf_access_token_env = os.environ.get("HUGGINGFACE_ACCESS_TOKEN")

In [6]:
from langchain import HuggingFaceHub
hf_llm = HuggingFaceHub(huggingfacehub_api_token = hf_access_token_env,
                        repo_id="google/flan-t5-large",
                        model_kwargs={"temperature" : 0,"max_length":64})

d:\Langchain\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
d:\Langchain\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
output = hf_llm.predict("What color is an apple ?")
output

'green'

### Prompt templates & LLMChain

In [8]:
from langchain.prompts import PromptTemplate
promptTemplate = PromptTemplate(input_variables=['country'],
                                template= "Tell me the currency of {country}")

promptTemplate.format(country="Italy")

'Tell me the currency of Italy'

In [9]:
from langchain.chains import LLMChain
chain = LLMChain(llm = openai_llm,
                 prompt = promptTemplate)
chain.run("New Zeland")

d:\Langchain\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nThe currency of New Zealand is the New Zealand dollar (NZD).'

### Combining multiple chains using Simple Sequential Chain

In [10]:
captial_template = PromptTemplate(input_variables=['Country'],
                                template="Tell me the capital of the {Country}")

capital_chain = LLMChain(llm=openai_llm,prompt=captial_template)

famous_template = PromptTemplate(input_variables=['Capital'],
                                 template="suggest me 3 places to visit in {Capital}")

famous_chain = LLMChain(llm=openai_llm, prompt=famous_template)

In [11]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
print(chain.run("Greece"))

 Some popular places to visit in Athens are:

1. Acropolis of Athens: This ancient citadel is home to the iconic Parthenon temple and offers stunning views of the city. It is a must-visit for history and architecture enthusiasts.

2. Plaka neighborhood: Located at the foot of the Acropolis, this charming neighborhood is known for its picturesque streets, traditional tavernas, and souvenir shops. It is a great place to explore and experience the local culture.

3. National Archaeological Museum: This is one of the largest and most important museums in Greece, housing a vast collection of artifacts and treasures from various eras of Greek history. It is a must-visit for anyone interested in ancient Greek civilization.


### SequentialChain

In [12]:
from langchain.chains import SequentialChain

capital_template=PromptTemplate(input_variables=['country'],
                                template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=openai_llm,prompt=capital_template,output_key="capital")

In [13]:
currency_template = PromptTemplate(input_variables=['country'],
                                template="tell me the currency of {country}")

currency_chain =LLMChain(llm=openai_llm,prompt=currency_template,output_key="currency")

In [14]:

famous_template=PromptTemplate(input_variables=['capital'],
                               template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=openai_llm,prompt=famous_template,output_key="places")

In [15]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,currency_chain,famous_chain],
                      input_variables=['country'],
                      output_variables=['capital','currency',"places"])

In [16]:
chain({'country':"Japan"})

d:\Langchain\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'Japan',
 'capital': '\n\nThe capital of Japan is Tokyo.',
 'currency': '\n\nThe currency of Japan is the Japanese yen (¥ or JPY).',
 'places': ' Here are some amazing places to visit in Tokyo:\n\n1. Tokyo Skytree\n2. Sensoji Temple\n3. Tsukiji Fish Market\n4. Meiji Shrine\n5. Shibuya Crossing\n6. Imperial Palace\n7. Akihabara\n8. Shinjuku Gyoen National Garden\n9. Tokyo Disneyland and DisneySea\n10. Harajuku\n11. Odaiba\n12. Ueno Park\n13. Roppongi Hills\n14. Tokyo Tower\n15. Ginza\n16. Tokyo National Museum\n17. Asakusa\n18. Tokyo Dome City\n19. Tokyo Sea Life Park\n20. Tokyo Metropolitan Government Building Observatory.'}

### Chatmodels with ChatOpenAI

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [34]:
openai_chat_llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],model_name="gpt-3.5-turbo",temperature=0.6)

In [35]:
openai_chat_llm([
    SystemMessage(content = "You are a Chef AI assistent"),
    HumanMessage(content = "Please provide me some cooking tips for a candle light dinner")
])

AIMessage(content="Of course! Here are some cooking tips for a candlelight dinner:\n\n1. Choose recipes that are not too complicated or time-consuming, so you can spend more time enjoying the meal with your guest.\n2. Prep as much as you can ahead of time to minimize stress during the actual cooking process.\n3. Consider dishes that can be easily plated and served elegantly, such as a well-presented salad or a beautifully arranged charcuterie board.\n4. Opt for foods that are known aphrodisiacs, such as oysters, chocolate, and strawberries, to set the mood for a romantic evening.\n5. Don't forget to pay attention to the presentation of the dishes – garnishes, sauces, and plating can make a big difference in creating a memorable dining experience.\n6. Pair your dishes with a complementary wine or cocktail to enhance the flavors and overall dining experience.\n7. Lastly, don't stress too much about the cooking process – the most important thing is to create a warm and inviting atmosphere

### PromptTemplate + LLM + Output Parser

In [56]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [87]:
class CommanSepratedOutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [97]:
template = "You are a asistent who has to give 5 words which defines the input word without numbering the 5 words"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [98]:
chain=chatprompt|openai_chat_llm|CommanSepratedOutput()

In [100]:
chain.invoke({"text":"Tiger"})

['Fierce', ' powerful', ' majestic', ' carnivorous', ' striped']